In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

ds = load_dataset("MARIO-Math-Reasoning/AlphaMath-Trainset")

README.md:   0%|          | 0.00/822 [00:00<?, ?B/s]

round3_training_data.json:   0%|          | 0.00/215M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/115551 [00:00<?, ? examples/s]

In [3]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['answer', 'id', 'output', 'instruction', 'input'],
        num_rows: 115551
    })
})


In [4]:
import pandas as pd 
train_df=pd.DataFrame(ds["train"])


In [5]:
train_df.head()

,answer,id,output,instruction,input
0,234,gsm8k_525,"[{""step"": ""<step>\n<p>\nTo find how many books...",<question> There are 336 books in a library. O...,
1,234,gsm8k_525,"[{""step"": ""<step>\n<p>\nTo find how many books...",<question> There are 336 books in a library. O...,
2,234,gsm8k_525,"[{""step"": ""<step>\n<p>\nTo calculate how many ...",<question> There are 336 books in a library. O...,
3,234,gsm8k_525,"[{""step"": ""<step>\n<p>\nTo find the number of ...",<question> There are 336 books in a library. O...,
4,234,gsm8k_525,"[{""step"": ""<step>\n<p>\nTo solve this problem,...",<question> There are 336 books in a library. O...,


In [6]:
import pandas as pd
import json

# Sample DataFrame containing the column with the string

# Function to parse JSON-like strings into columns
def parse_json_column(json_str):
    # Parse the string into a Python list of dictionaries
    json_data = json.loads(json_str)
    
    # Convert the list of dictionaries into a DataFrame
    return pd.DataFrame(json_data)

# Apply the function to each row of the DataFrame and expand into separate columns
expanded_df = train_df['output'].apply(parse_json_column)

# Concatenate the list of DataFrames
final_df = pd.concat(expanded_df.tolist(), ignore_index=True)

# Show the final DataFrame with separated columns
print(final_df)


                                                     step         P         Q  \
0       <step>\n<p>\nTo find how many books are there ...  0.997700  0.991211   
1       <step>\n<p>\nAfter performing the calculation,...  0.950430  1.000000   
2       <step>\n<p>\nTo find how many books are there ...  0.997700  0.991211   
3       <step>\n<p>\nI have calculated the number of b...  0.998441  1.000000   
4       <step>\n<p>\nTo calculate how many books there...  0.972948  0.988281   
...                                                   ...       ...       ...   
309322  <step>\n<p>\nIt seems that the code is not wor...  0.999806 -0.878906   
309323  <step>\n<p>\nIt seems that the code is not wor...  0.935737 -1.000000   
309324  <step>\n<p>\nTo find the sum of the $2007$ roo...  0.932510 -0.795796   
309325  <step>\n<p>\nThe `sum` function is not availab...  0.975516 -0.976128   
309326  <step>\n<p>\nFrom the result, we can see that ...  0.989254 -1.000000   

        depth  
0          

In [7]:
to_train_df=final_df[1:50000]
to_test_df=final_df[51000:52000]

In [8]:
to_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 1 to 49999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   step    49999 non-null  object 
 1   P       49999 non-null  float64
 2   Q       49999 non-null  float64
 3   depth   49999 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 1.5+ MB


In [ ]:
import torch
from transformers import BartForSequenceClassification, BartTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
import pandas as pd

# Load your dataset (ensure that 'final_df' has the necessary columns like 'step' and 'P')
to_train_df # Assuming final_df is already loaded and processed

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(to_train_df)
eval_dataset=Dataset.from_pandas(to_test_df)

# Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Tokenize the input texts
def tokenize_function(example):
    return tokenizer(example['step'], padding='max_length', truncation=True, max_length=1024)

# Apply tokenization with padding and truncation
tokenized_dataset = dataset.map(tokenize_function, batched=True)
eval_dataset=eval_dataset.map(tokenize_function, batched=True)
# Load the pre-trained BART model and modify for regression
class BartForRegression(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bart = BartForSequenceClassification.from_pretrained('facebook/bart-base', num_labels=1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bart(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = None
        if labels is not None:
            loss_fct = torch.nn.MSELoss()
            loss = loss_fct(logits.view(-1), labels.view(-1))
        return {'loss': loss, 'logits': logits}

# Instantiate the model
model = BartForRegression()

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,              # Adjust this as needed
    per_device_train_batch_size=4,   # Adjust batch size as needed
    evaluation_strategy="epoch",     # Evaluate after each epoch
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=1000,
)

# Convert the 'P' column to a float tensor
def add_labels(example):
    example['labels'] = torch.tensor(example['P'], dtype=torch.float)
    return example

# Apply the label transformation
tokenized_dataset = tokenized_dataset.map(add_labels)
eval_dataset=eval_dataset.map(add_labels)
print("Tokenized input length: ", len(tokenized_dataset[0]['input_ids']))



# Remove unnecessary columns that the model doesn't need for training
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset=eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# Use DataCollatorWithPadding to handle padding dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator  # Ensure proper padding
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model("./trained_bart_regression")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Tokenized input length:  1024


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


In [ ]:
heeee

## try 2

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [ ]:
notebook_login()

In [ ]:
# Load dataset
dataset = load_dataset(dataset_id)

# Training and testing datasets
train_dataset = dataset['train']
test_dataset = dataset["test"].shard(num_shards=2, index=0)

# Validation dataset
val_dataset = dataset['test'].shard(num_shards=2, index=1)

# Preprocessing
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

# This function tokenizes the input text using the RoBERTa tokenizer. 
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=4096)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))


In [ ]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# # We will need this to directly output the class names when using the pipeline without mapping the labels later.
# # Extract the number of classes and their names
# num_labels = dataset['train'].features['label'].num_classes
# class_names = dataset["train"].features["label"].names
# print(f"number of labels: {num_labels}")
# print(f"the labels: {class_names}")

# # Create an id2label mapping
# id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [ ]:
from transformers import LongformerConfig, LongformerModel


configuration = LongformerConfig()


model = LongformerModel(configuration)


configuration = model.config

In [ ]:
# model_id = "roberta-base"
# dataset_id = "ag_news"
# relace the value with your model: ex <hugging-face-user>/<model-name>
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 
repository_id = "/kaggle/working/"
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
#     report_to="tensorboard",
#     push_to_hub=True,
#     hub_strategy="every_save",
#     hub_model_id=repository_id,
#     hub_token=HfFolder.get_token(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1


In [ ]:
tokenized_dataset = tokenized_dataset.map(add_labels)
eval_dataset=eval_dataset.map(add_labels)

In [ ]:
print(tokenized_dataset )

In [ ]:
eval_dataset=Dataset.from_pandas(to_test_df)
eval_dataset=eval_dataset.map(tokenize_function, batched=True)
eval_dataset=eval_dataset.map(add_labels)
print(eval_dataset)

In [ ]:
print(eval_dataset[0])

In [ ]:
torch.manual_seed(42)